# Step 1: Import the required libraries

In [ ]:
#!pip install numpy rasterio scikit-learn matplotlib


# Step 2: Reading the Sentinel-2 Image
Assuming that you have already preprocessed the Sentinel-2 image and saved it as a GeoTIFF file, the next step is to read this file using rasterio.

# Step 3: Applying K-means Clustering
You will then apply the K-means clustering algorithm to the image data to classify it into different land cover types

# Step 4: Evaluating clusters with the Elbow method and Silhouette Index

In [30]:
import rasterio
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [32]:
# Path to your preprocessed Sentinel-2 image
image_path = 'Data/resampled_output.tif'

# Read the raster data
with rasterio.open(image_path) as src:
    img = src.read([1, 2, 3, 4, 5, 6])  # Assuming band order in TIFF matches desired
    transform = src.transform

# Reshape and transpose data
img = np.transpose(img, (1, 2, 0))
pixels = img.reshape(-1, img.shape[2])

# Handling NaNs
pixels = np.nan_to_num(pixels)

# Range of potential clusters
cluster_range = range(2, 11)

# Initialize lists to store the evaluations
wss = []
silhouette_scores = []

for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(pixels)
    
    # Within-cluster sum of squares (WSS)
    wss.append(kmeans.inertia_)
    
    # Compute the silhouette scores
    silhouette_avg = silhouette_score(pixels, labels)
    silhouette_scores.append(silhouette_avg)

    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")



KeyboardInterrupt: 

In [ ]:
# Plotting the Elbow and Silhouette scores
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Number of Clusters')
ax1.set_ylabel('WSS (Elbow Method)', color=color)
ax1.plot(cluster_range, wss, marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('Silhouette Score', color=color)
ax2.plot(cluster_range, silhouette_scores, marker='o', color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.title('Elbow and Silhouette Analysis for Determining Optimal Clusters')
plt.show()

# Re-run K=means with Optimal Clusters

Once we get the optimal number of clusters, as previously determined by the silhouette scor and WSS values, we need to re-run the K-means clustering with the chosen number of clusters to get the classification labels for the entire dataset. This time, store the labels and use them to create an image.

In [ ]:
# Optimal number of clusters (determined from previous analysis)
optimal_clusters = 5 # replace this with the optimal number determined previously
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
labels = kmeans.fit_predict(pixels)


# Reshape the Labels into an image

In [ ]:
# Reshape labels to the image shape
classified_image = labels.reshape(img.shape[0], img.shape[1])

# Plotting the classified image
plt.figure(figsize=(10, 10))
plt.imshow(classified_image, cmap='viridis') # can choose different colour map
plt.colorbar(label='Cluster Labels') # To help understand which color corresponds to which cluster label
plt.title('K-Means Classification Result')
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()
